In [3]:
import pandas as pd

import sys, os, requests, json, datetime
sys.path.insert(1, '../../../../scripts/')
from s3_support import *

In [4]:
# pulling random forms
q = '''select ad.*, qd.*
            from analytics_daily as ad
                left join analyticsqgiv_daily as qd on ad.form=qd.form and ad.date=qd.date
            where
                ad.form in (select form
                        from analyticsqgiv_daily
                        group by form
                            order by random() limit 100) and
                ad.date=(select max(date) from analytics_daily) and
                ad.product=1
            limit 100;'''
test_forms_data = redshift_query_read(q)
test_forms_data = test_forms_data.loc[:,~test_forms_data.columns.duplicated()]

In [5]:
test_forms_data.head(3)

,date,org,form,product,vt_trans_count,don_form_trans_count,kiosk_trans_count,p2p_trans_count,mobile_trans_count,mobilevt_trans_count,...,collect_address_mobile,enable_donorlogins,enable_sms,new_rec_volume,new_rec_count,reg_count,dl_trans_volume,dl_trans_count,dl_new_rec_count,dl_new_rec_volume
0,2020-04-11,444297,953734,1,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0.0,0,0,0.0
1,2020-04-11,443553,955030,1,0,0,0,0,0,0,...,0,0,1,0.0,0,0,0.0,0,0,0.0
2,2020-04-11,442974,937442,1,0,0,0,0,0,0,...,0,0,1,0.0,0,0,0.0,0,0,0.0


In [6]:
# getting widget data
q_created = '''select
            users.org as org,
            count(created_widget.id) as widgets_created,
            date_trunc('day', created_widget.original_timestamp) as day
        from created_widget
            left join users on created_widget.uuid=users.uuid
        group by day, org'''
q_deleted = '''select
            users.org as org,
            count(deleted_widget.id) as widgets_deleted,
            date_trunc('day', deleted_widget.original_timestamp) as day
        from deleted_widget
            left join users on deleted_widget.uuid=users.uuid
        group by day, org'''
widgets_df = redshift_query_read(q_created, schema="secure")
widgets_df = widgets_df.merge(redshift_query_read(q_deleted, schema="secure"), on=['org', 'day'])
widgets_df = widgets_df[widgets_df['org'].isin(test_forms_data['org'].tolist())]
widgets_data = None
for org in widgets_df['org'].tolist():
    this_df = widgets_df[widgets_df['org']==org].copy().sort_values('day', ascending=True)
    this_df['created_cumsum'] = this_df['widgets_created'].cumsum()
    this_df['deleted_cumsum'] = this_df['widgets_deleted'].cumsum()
    this_df['widgets'] = this_df['created_cumsum'] - this_df['deleted_cumsum']

    if widgets_data is None:
        widgets_data = this_df[['org', 'day', 'widgets']]
    else:
        widgets_data = widgets_data.append(widgets_data)
widgets_df = widgets_data

In [7]:
widgets_df.head(3)

AttributeError: 'NoneType' object has no attribute 'head'

In [8]:
# cms
q = '''select
            users.org as org,
            count(saved_page.id) as page_updates,
            date_trunc('day', saved_page.original_timestamp) as day
        from saved_page
            left join users on saved_page.uuid=users.uuid
        group by day, org'''
cms_df = redshift_query_read(q, schema="secure")
cms_df = cms_df[cms_df['org'].isin(test_forms_data['org'].tolist())]
cms_data = None
for org in cms_df['org'].tolist():
    this_df = cms_data[cms_data['org']==org].copy().sort_values('day', ascending=True)
    this_df['page_updates_cumsum'] = this_df['page_updates'].cumsum()

    if cms_data is None:
        cms_data = this_df
    else:
        cms_data = cms_data.append(this_df)
cms_df = cms_data

In [9]:
cms_df.head(3)

AttributeError: 'NoneType' object has no attribute 'head'

In [10]:
# goals
q_created = '''select
            users.org as org,
            count(gs.id) as goals_saved,
            date_trunc('day', gs.original_timestamp) as day
        from goals_saved_item_settings as gs
            left join users on gs.uuid=users.uuid
        group by day, org'''
q_deleted = '''select
            users.org as org,
            count(gs.id) as goals_deleted,
            date_trunc('day', gs.original_timestamp) as day
        from goals_deleted_item as gs
            left join users on gs.uuid=users.uuid
        group by day, org'''
goals_df = redshift_query_read(q_created, schema="secure")
goals_df = goals_df.merge(redshift_query_read(q_deleted, schema="secure"), on=['org', 'day'])
goals_df = goals_df[goals_df['org'].isin(test_forms_data['org'].tolist())]
goals_data = None
for org in goals_df['org'].tolist():
    this_df = goals_df[goals_df['org']==org].copy().sort_values('day', ascending=True)
    this_df['created_cumsum'] = this_df['goals_saved'].cumsum()
    this_df['deleted_cumsum'] = this_df['goals_deleted'].cumsum()
    this_df['goals'] = this_df['created_cumsum'] - this_df['deleted_cumsum']

    if goals_data is None:
        goals_data = this_df[['org', 'day', 'goals']]
    else:
        goals_data = goals_data.append(this_df)
goals_df = goals_data

In [11]:
goals_df.head(3)

AttributeError: 'NoneType' object has no attribute 'head'

In [12]:
# formatting input
test_form_params_data = []
for _, r in test_forms_data.iterrows():
    if widgets_df is not None:
        widgets_created = widgets_df[widgets_df['org']==r['org']].iloc[-1]
    else:
        widgets_created = 0
    if cms_df is not None:
        cms_updates = cms_df[cms_df['org']==r['org']].iloc[-1]
    else:
        cms_updates = 0
    if goals_df is not None:
        has_goals = goals_df[goals_df['org']==r['org']].iloc[-1] > 0
    else:
        has_goals = False
    has_widgets = widgets_created != 0
    cms_active = cms_updates != 0

    test_form_params_data.append({
        'form': r['form'],
        'settings': {
            'pledge_active': r['pledge_active'],
            'has_min_amount': r['min_amount'] != 0.,
            'has_max_amount': r['max_amount'] != 0,
            'permit_anonymous': r['permit_anonymous'],
            'donation_active': r['donation_active'],
            'multirestriction_system': r['multirestriction_system'],
            'collect_company': r['collect_company'],
            'collect_phone': r['collect_phone'],
            'collect_optin': r['collect_optin'],
            'collect_address_mobile': r['collect_address_mobile'],
            'enable_donorlogins': r['enable_donorlogins'],
            'enable_sms': r['enable_sms'],
            'has_widgets': has_widgets,
            'cms_active': cms_active,
            'has_goals': has_goals
        },
        'features': {
            'restrictions': r['restrictions'],
            'amounts': r['amounts'],
            'opt_fields': r['opt_fields'],
            'req_fields': r['req_fields'],
            'min_amount': r['min_amount'],
            'max_amount': r['max_amount'],
            'cms_updates': cms_updates,
            'widgets_created': widgets_created
        }
    })

In [15]:
len(test_form_params_data), test_form_params_data[0]

(85,
 {'form': 953734,
  'settings': {'pledge_active': 0,
   'has_min_amount': False,
   'has_max_amount': False,
   'permit_anonymous': 0,
   'donation_active': 0,
   'multirestriction_system': 0,
   'collect_company': 0,
   'collect_phone': 0,
   'collect_optin': 2,
   'collect_address_mobile': 0,
   'enable_donorlogins': 0,
   'enable_sms': 0,
   'has_widgets': False,
   'cms_active': False,
   'has_goals': False},
  'features': {'restrictions': 0,
   'amounts': 4,
   'opt_fields': 0,
   'req_fields': 0,
   'min_amount': 0.0,
   'max_amount': 0.0,
   'cms_updates': 0,
   'widgets_created': 0}})